# Gist Database Project

This project interfaces with GitHub's Gist API to fetch, store, and search Gist data. Using Python libraries like `requests` and `sqlite3`, we build a powerful Gist database with flexible search capabilities.


## Step 1: Database Initialization

We start by setting up a SQLite database and creating the required schema to store Gist information.


In [1]:
!pip install requests

# Import libraries
import sqlite3
import requests
from datetime import datetime
import json

# Initialize database connection
db = sqlite3.connect(":memory:")

# Function to create the database schema
def initialize_db(db_connection):
    cursor = db_connection.cursor()
    cursor.executescript("""
    DROP TABLE IF EXISTS gists;

    CREATE TABLE gists (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        github_id TEXT NOT NULL UNIQUE,
        description TEXT,
        public INTEGER,
        created_at TEXT,
        updated_at TEXT,
        url TEXT
    );
    """)
    db_connection.commit()
    cursor.close()
    print("Database schema created successfully!")

# Initialize the database with schema
initialize_db(db)


Database schema created successfully!


Step 2: Fetching Gists from GitHub


In [2]:
# Function to fetch Gists from GitHub for a specific user
def fetch_gists(username):
    # GitHub API endpoint to fetch user's Gists
    url = f"https://api.github.com/users/{username}/gists"

    try:
        # Make the GET request to the API
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            gists = response.json()
            print(f"Successfully fetched {len(gists)} Gists for user: {username}")
            return gists
        else:
            # Handle cases where the API request fails
            print(f"Failed to fetch Gists for user: {username}")
            print(f"Status Code: {response.status_code}, Message: {response.text}")
            return []
    except requests.RequestException as e:
        print(f"An error occurred while fetching Gists: {str(e)}")
        return []

sample_user = 'matiascaputti'
gists = fetch_gists(sample_user)

# Display the first Gist (if available) to verify output
if gists:
    print(json.dumps(gists[0], indent=4))


Successfully fetched 6 Gists for user: matiascaputti
{
    "url": "https://api.github.com/gists/9e58a3ca9cc0a4ec44dc1b0a5a45b8f7",
    "forks_url": "https://api.github.com/gists/9e58a3ca9cc0a4ec44dc1b0a5a45b8f7/forks",
    "commits_url": "https://api.github.com/gists/9e58a3ca9cc0a4ec44dc1b0a5a45b8f7/commits",
    "id": "9e58a3ca9cc0a4ec44dc1b0a5a45b8f7",
    "node_id": "G_kwDOAGvPOdoAIDllNThhM2NhOWNjMGE0ZWM0NGRjMWIwYTVhNDViOGY3",
    "git_pull_url": "https://gist.github.com/9e58a3ca9cc0a4ec44dc1b0a5a45b8f7.git",
    "git_push_url": "https://gist.github.com/9e58a3ca9cc0a4ec44dc1b0a5a45b8f7.git",
    "html_url": "https://gist.github.com/matiascaputti/9e58a3ca9cc0a4ec44dc1b0a5a45b8f7",
    "files": {
        "dom3d.js": {
            "filename": "dom3d.js",
            "type": "text/javascript",
            "language": "JavaScript",
            "raw_url": "https://gist.githubusercontent.com/matiascaputti/9e58a3ca9cc0a4ec44dc1b0a5a45b8f7/raw/bf7cab8bd643a428fb4fcd73f47bd4b8e1b75c6b/dom3d.j

Step 3: Importing Gists into the Database


In [3]:
def import_gists_to_database(db_connection, username, commit=True):
    # Fetch Gists for the provided username
    gists = fetch_gists(username)

    if not gists:
        print(f"No Gists found for user: {username}")
        return

    cursor = db_connection.cursor()

    # Iterate over each Gist and insert relevant data into the database
    for gist in gists:
        # Extract required details
        github_id = gist['id']
        description = gist['description'] if gist['description'] else "No description"
        public = 1 if gist['public'] else 0
        created_at = gist['created_at']
        updated_at = gist['updated_at']
        url = gist['html_url']

        # Insert or replace the Gist details into the database
        cursor.execute("""
            INSERT OR REPLACE INTO gists (github_id, description, public, created_at, updated_at, url)
            VALUES (?, ?, ?, ?, ?, ?)
        """, (github_id, description, public, created_at, updated_at, url))

    # Commit changes if the commit flag is True
    if commit:
        db_connection.commit()
        print(f"Successfully imported {len(gists)} Gists for user: {username} into the database.")

    cursor.close()

In [4]:
import_gists_to_database(db, sample_user)

# Check that data has been inserted correctly
def display_all_gists(db_connection):
    cursor = db_connection.cursor()
    cursor.execute("SELECT * FROM gists")
    rows = cursor.fetchall()
    cursor.close()
    for row in rows:
        print(row)

display_all_gists(db)

Successfully fetched 6 Gists for user: matiascaputti
Successfully imported 6 Gists for user: matiascaputti into the database.
(1, '9e58a3ca9cc0a4ec44dc1b0a5a45b8f7', '3D DOM viewer, copy-paste this into your console to visualise the DOM topographically.', 1, '2024-08-06T20:45:52Z', '2024-08-06T20:45:52Z', 'https://gist.github.com/matiascaputti/9e58a3ca9cc0a4ec44dc1b0a5a45b8f7')
(2, '72d6110ca2bb4a8058b3fb6b73b21503', 'No description', 1, '2018-11-19T20:08:03Z', '2018-11-19T20:08:04Z', 'https://gist.github.com/matiascaputti/72d6110ca2bb4a8058b3fb6b73b21503')
(3, '8d774e49fa66b1991056161ca26fc3ef', 'No description', 1, '2018-11-19T20:07:12Z', '2018-11-19T20:07:12Z', 'https://gist.github.com/matiascaputti/8d774e49fa66b1991056161ca26fc3ef')
(4, '0922f7ffa84d56ac602798fef2fa0f16', 'Fix "Permission denied (publickey)" error when pushing with Git', 1, '2018-09-17T02:21:28Z', '2018-09-17T02:21:29Z', 'https://gist.github.com/matiascaputti/0922f7ffa84d56ac602798fef2fa0f16')
(5, 'a426b244ea30adf3

Step 4: Implementing Search Functionality


In [5]:
def search_gists(db_connection, github_id=None, created_at=None):
    cursor = db_connection.cursor()

    # Base query
    query = "SELECT * FROM gists WHERE 1=1"
    params = {}

    # Append filters based on optional arguments
    if github_id:
        query += " AND github_id = :github_id"
        params['github_id'] = github_id

    if created_at:
        query += " AND datetime(created_at) = datetime(:created_at)"
        params['created_at'] = created_at

    # Execute the constructed query
    cursor.execute(query, params)
    rows = cursor.fetchall()
    cursor.close()

    # Return a list of Gist objects (simple dictionary here for representation)
    return [
        {
            'id': row[0],
            'github_id': row[1],
            'description': row[2],
            'public': row[3],
            'created_at': row[4],
            'updated_at': row[5],
            'url': row[6]
        }
        for row in rows
    ]

In [6]:
all_gists = search_gists(db)
print(f"All Gists: {all_gists}")

if all_gists:
    sample_github_id = all_gists[0]['github_id']
    print(f"\nSearching for Gist with GitHub ID: {sample_github_id}")
    gist_by_id = search_gists(db, github_id=sample_github_id)
    print(gist_by_id)

if all_gists:
    sample_created_at = all_gists[0]['created_at']
    print(f"\nSearching for Gist created at: {sample_created_at}")
    gists_by_date = search_gists(db, created_at=sample_created_at)
    print(gists_by_date)

All Gists: [{'id': 1, 'github_id': '9e58a3ca9cc0a4ec44dc1b0a5a45b8f7', 'description': '3D DOM viewer, copy-paste this into your console to visualise the DOM topographically.', 'public': 1, 'created_at': '2024-08-06T20:45:52Z', 'updated_at': '2024-08-06T20:45:52Z', 'url': 'https://gist.github.com/matiascaputti/9e58a3ca9cc0a4ec44dc1b0a5a45b8f7'}, {'id': 2, 'github_id': '72d6110ca2bb4a8058b3fb6b73b21503', 'description': 'No description', 'public': 1, 'created_at': '2018-11-19T20:08:03Z', 'updated_at': '2018-11-19T20:08:04Z', 'url': 'https://gist.github.com/matiascaputti/72d6110ca2bb4a8058b3fb6b73b21503'}, {'id': 3, 'github_id': '8d774e49fa66b1991056161ca26fc3ef', 'description': 'No description', 'public': 1, 'created_at': '2018-11-19T20:07:12Z', 'updated_at': '2018-11-19T20:07:12Z', 'url': 'https://gist.github.com/matiascaputti/8d774e49fa66b1991056161ca26fc3ef'}, {'id': 4, 'github_id': '0922f7ffa84d56ac602798fef2fa0f16', 'description': 'Fix "Permission denied (publickey)" error when pushi

## Step 5: Demonstration of All Functionalities

In this section, we demonstrate how to:
1. Fetch Gists from GitHub and import them into the database.
2. Search Gists with various filters.
3. Use date-based filters for more advanced searches.


In [7]:
def search_gists(
    db_connection,
    github_id=None,
    created_at=None,
    created_at__gt=None,
    created_at__gte=None,
    created_at__lt=None,
    created_at__lte=None,
    updated_at__gt=None,
    updated_at__gte=None,
    updated_at__lt=None,
    updated_at__lte=None
):
    cursor = db_connection.cursor()

    # Base query
    query = "SELECT * FROM gists WHERE 1=1"
    params = {}

    # Append filters based on optional arguments
    if github_id:
        query += " AND github_id = :github_id"
        params['github_id'] = github_id

    if created_at:
        query += " AND datetime(created_at) = datetime(:created_at)"
        params['created_at'] = created_at

    # Date filters for created_at
    if created_at__gt:
        query += " AND datetime(created_at) > datetime(:created_at__gt)"
        params['created_at__gt'] = created_at__gt

    if created_at__gte:
        query += " AND datetime(created_at) >= datetime(:created_at__gte)"
        params['created_at__gte'] = created_at__gte

    if created_at__lt:
        query += " AND datetime(created_at) < datetime(:created_at__lt)"
        params['created_at__lt'] = created_at__lt

    if created_at__lte:
        query += " AND datetime(created_at) <= datetime(:created_at__lte)"
        params['created_at__lte'] = created_at__lte

    # Date filters for updated_at
    if updated_at__gt:
        query += " AND datetime(updated_at) > datetime(:updated_at__gt)"
        params['updated_at__gt'] = updated_at__gt

    if updated_at__gte:
        query += " AND datetime(updated_at) >= datetime(:updated_at__gte)"
        params['updated_at__gte'] = updated_at__gte

    if updated_at__lt:
        query += " AND datetime(updated_at) < datetime(:updated_at__lt)"
        params['updated_at__lt'] = updated_at__lt

    if updated_at__lte:
        query += " AND datetime(updated_at) <= datetime(:updated_at__lte)"
        params['updated_at__lte'] = updated_at__lte

    # Execute the constructed query
    cursor.execute(query, params)
    rows = cursor.fetchall()
    cursor.close()

    # Return a list of Gist objects (simple dictionary here for representation)
    return [
        {
            'id': row[0],
            'github_id': row[1],
            'description': row[2],
            'public': row[3],
            'created_at': row[4],
            'updated_at': row[5],
            'url': row[6]
        }
        for row in rows
    ]

In [8]:
# Test the enhanced search function with date filters
from datetime import datetime, timedelta

# Fetch all Gists again to use for testing
all_gists = search_gists(db)

if all_gists:
    # Set a sample date range for testing (change these as needed)
    some_date = all_gists[0]['created_at']
    date_format = "%Y-%m-%dT%H:%M:%SZ"

    # Convert string to datetime
    sample_datetime = datetime.strptime(some_date, date_format)

    # Test created_at greater than
    print(f"\nSearching for Gists created after: {sample_datetime}")
    gists_created_after = search_gists(db, created_at__gt=sample_datetime)
    print(gists_created_after)

    # Test created_at less than
    print(f"\nSearching for Gists created before: {sample_datetime}")
    gists_created_before = search_gists(db, created_at__lt=sample_datetime)
    print(gists_created_before)




Searching for Gists created after: 2024-08-06 20:45:52
[]

Searching for Gists created before: 2024-08-06 20:45:52
[{'id': 2, 'github_id': '72d6110ca2bb4a8058b3fb6b73b21503', 'description': 'No description', 'public': 1, 'created_at': '2018-11-19T20:08:03Z', 'updated_at': '2018-11-19T20:08:04Z', 'url': 'https://gist.github.com/matiascaputti/72d6110ca2bb4a8058b3fb6b73b21503'}, {'id': 3, 'github_id': '8d774e49fa66b1991056161ca26fc3ef', 'description': 'No description', 'public': 1, 'created_at': '2018-11-19T20:07:12Z', 'updated_at': '2018-11-19T20:07:12Z', 'url': 'https://gist.github.com/matiascaputti/8d774e49fa66b1991056161ca26fc3ef'}, {'id': 4, 'github_id': '0922f7ffa84d56ac602798fef2fa0f16', 'description': 'Fix "Permission denied (publickey)" error when pushing with Git', 'public': 1, 'created_at': '2018-09-17T02:21:28Z', 'updated_at': '2018-09-17T02:21:29Z', 'url': 'https://gist.github.com/matiascaputti/0922f7ffa84d56ac602798fef2fa0f16'}, {'id': 5, 'github_id': 'a426b244ea30adf3ea715

## Conclusion

In this project, we successfully built a database that interfaces with GitHub's Gist API. We demonstrated how to:
- Fetch Gists from GitHub.
- Store Gists in a SQLite database.
- Perform advanced searches on the Gist data.

This project showcases the power of combining APIs with databases to build useful and flexible applications.
